Cell 1: Install dependencies

In [ ]:
!pip install pandas scikit-learn joblib matplotlib


Cell 2: Load model predictions

In [ ]:
import pandas as pd

# Load individual model outputs (0–1 probability or raw scores)
xgb = pd.read_csv('/content/xgb_preds.csv')
lstm = pd.read_csv('/content/lstm_preds.csv')
finbert = pd.read_csv('/content/finbert_preds.csv')
labels = pd.read_csv('/content/true_labels.csv')  # columns: ['date', 'true']

# Merge everything
df = xgb.merge(lstm, on='date', suffixes=('_xgb', '_lstm'))
df = df.merge(finbert, on='date')
df = df.rename(columns={'pred': 'pred_finbert'})
df = df.merge(labels, on='date')


Cell 3: Train Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = df[['pred_xgb', 'pred_lstm', 'pred_finbert']]
y = df['true']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

bayes_model = GaussianNB()
bayes_model.fit(X_train, y_train)

y_pred = bayes_model.predict(X_test)
y_prob = bayes_model.predict_proba(X_test)

print(classification_report(y_test, y_pred))


Cell 4: Visualize confidence

In [ ]:
import matplotlib.pyplot as plt

plt.hist(y_prob[:, 1], bins=20, alpha=0.7)
plt.title("Bayesian Model Confidence (Class 1: UP)")
plt.xlabel("Confidence Score")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()


Cell 5: Save model

In [ ]:
import joblib
joblib.dump(bayes_model, "bayesian_net_model.pkl")
print("✅ Bayesian model saved.")
